<a href="https://colab.research.google.com/github/aaronshi2017/pytorch-Deep-Learning/blob/master/3_3_Linear_Regression_simplied_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import torch
from torch.utils import data
!pip install d2l
from d2l import torch as d2l

#define the true value of w (weight权重) and b (bias 偏置)并换成张量形式
true_w=torch.tensor([2,-3.4]) 
true_b=4.2
features,labels=d2l.synthetic_data(true_w,true_b, 1000) #生成线性回归训练集 refer to following original code for synthetic_data function, features is X, labels is y, they are generated based on givein w and b values


     |████████████████████████████████| 83 kB 1.2 MB/s 


In [ ]:
# # Defined in file: ./chapter_linear-networks/linear-regression-scratch.md
# [docs]def synthetic_data(w, b, num_examples):
#     """Generate y = Xw + b + noise."""
#     X = d2l.normal(0, 1, (num_examples, len(w)))
#     y = d2l.matmul(X, w) + b
#     y += d2l.normal(0, 0.01, y.shape) #adding noise normal distribution with mean 0, variance 0.01
#     return X, d2l.reshape(y, (-1, 1))

In [4]:
def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器。"""
    dataset = data.TensorDataset(*data_arrays) #Dataset wrapping tensors.
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10 # use batch size =10 to do gradient descent algorithm
data_iter = load_array((features, labels), batch_size)

In [5]:
#https://discuss.pytorch.org/t/make-a-tensordataset-and-dataloader-with-multiple-inputs-parameters/26605/5
# Example how to build up Dataloader 
# nb_samples = 100
# features = torch.randn(nb_samples, 10)
# labels = torch.empty(nb_samples, dtype=torch.long).random_(10)
# adjacency = torch.randn(nb_samples, 5)
# laplacian = torch.randn(nb_samples, 7)

# dataset = TensorDataset(features, labels, adjacency, laplacian)
# loader = DataLoader(
#     dataset,
#     batch_size=2
# )

# for batch_idx, (x, y, a, l) in enumerate(loader):
#     print(x.shape, y.shape, a.shape, l.shape)

In [13]:
next(iter(data_iter)) # demo how to use next(iter) to iterate the data 

[tensor([[ 0.7477,  0.4514],
         [-0.8593,  0.3605],
         [ 1.5132,  0.4942],
         [-0.0871, -2.2118],
         [ 1.6509,  0.0791],
         [-1.7786,  0.9295],
         [ 0.7407,  0.6482],
         [ 0.0981,  0.1206],
         [ 1.5421,  0.1012],
         [ 0.6602,  0.2343]]), tensor([[ 4.1736],
         [ 1.2414],
         [ 5.5460],
         [11.5527],
         [ 7.1976],
         [-2.5218],
         [ 3.4618],
         [ 3.9671],
         [ 6.9314],
         [ 4.7202]])]

In [14]:
next(iter(data_iter))

[tensor([[-2.3749e-01,  1.9942e+00],
         [-5.3149e-01, -7.0952e-01],
         [-1.4470e-03, -1.2178e+00],
         [-6.6078e-01,  2.1827e-01],
         [ 6.2129e-01, -6.0856e-01],
         [ 1.3483e+00,  8.3115e-01],
         [ 2.2468e+00, -6.0395e-01],
         [ 1.2220e+00,  1.9590e-01],
         [ 1.4360e-01, -6.4285e-01],
         [ 1.9252e+00,  1.6720e+00]]), tensor([[-3.0746],
         [ 5.5519],
         [ 8.3276],
         [ 2.1283],
         [ 7.5343],
         [ 4.0535],
         [10.7319],
         [ 5.9621],
         [ 6.6655],
         [ 2.3577]])]

对于标准操作，我们可以使用框架的预定义好的层。这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。我们首先定义一个模型变量net，它是一个 Sequential 类的实例。 Sequential 类为串联在一起的多个层定义了一个容器。当给定输入数据， Sequential 实例将数据传入到第一层，然后将第一层的输出作为第二层的输入，依此类推。在下面的例子中，我们的模型只包含一个层，因此实际上不需要Sequential。但是由于以后几乎所有的模型都是多层的，在这里使用Sequential会让你熟悉标准的流水线。在 PyTorch 中，全连接层在 Linear 类中定义。值得注意的是，我们将两个参数传递到 nn.Linear 中。第一个指定输入特征形状，即 2，第二个指定输出特征形状，输出特征形状为单个标量，因此为 1。

In [7]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1)) # define all connected layers
net[0].weight.data.normal_(0,0.01) #visit the first layer net[0] and reset weight and bias
net[0].bias.data.fill_(0) # we initilize w and b with such values. Net denotes the neural network (sequential)


tensor([0.])

In [8]:
loss=nn.MSELoss()

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

https://www.geeksforgeeks.org/ml-stochastic-gradient-descent-sgd/
Stochastic Gradient Descent (SGD):
The word ‘stochastic‘ means a system or a process that is linked with a random probability. Hence, in Stochastic Gradient Descent, a few samples are selected randomly instead of the whole data set for each iteration. In Gradient Descent, there is a term called “batch” which denotes the total number of samples from a dataset that is used for calculating the gradient for each iteration. In typical Gradient Descent optimization, like Batch Gradient Descent, the batch is taken to be the whole dataset. Although, using the whole dataset is really useful for getting to the minima in a less noisy and less random manner, but the problem arises when our datasets gets big.
Suppose, you have a million samples in your dataset, so if you use a typical Gradient Descent optimization technique, you will have to use all of the one million samples for completing one iteration while performing the Gradient Descent, and it has to be done for every iteration until the minima is reached. Hence, it becomes computationally very expensive to perform.This problem is solved by Stochastic Gradient Descent. In SGD, it uses only a single sample, i.e., a batch size of one, to perform each iteration. The sample is randomly shuffled and selected for performing the iteration. SGD equals mini batch gradient descent algorithm with batch size=1.

One thing to be noted is that, as SGD is generally noisier than typical Gradient Descent, it usually took a higher number of iterations to reach the minima, because of its randomness in its descent. Even though it requires a higher number of iterations to reach the minima than typical Gradient Descent, it is still computationally much less expensive than typical Gradient Descent. Hence, in most scenarios, SGD is preferred over Batch Gradient Descent for optimizing a learning algorithm.

https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD

>>> optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

>>> optimizer.zero_grad()

>>> loss_fn(model(input), target).backward()

>>> optimizer.step()

Momentum [1] or SGD with momentum is method which helps accelerate gradients vectors in the right directions, thus leading to faster converging.
https://towardsdatascience.com/stochastic-gradient-descent-with-momentum-a84097641a5d
With Stochastic Gradient Descent we don’t compute the exact derivate of our loss function. Instead, we’re estimating it on a small batch. Which means we’re not always going in the optimal direction, because our derivatives are ‘noisy’. Just like in my graphs above. So, exponentially weighed averages can provide us a better estimate which is closer to the actual derivate than our noisy calculations. This is one reason why momentum might work better than classic SGD.


In [ ]:
# def SGD(f, theta0, alpha, num_iters):
#     """ 
#        Arguments:
#        f -- the function to optimize, it takes a single argument
#             and yield two outputs, a cost and the gradient
#             with respect to the arguments
#        theta0 -- the initial point to start SGD from
#        num_iters -- total iterations to run SGD for
#        Return:
#        theta -- the parameter value after SGD finishes
#     """
#     start_iter = 0
#     theta = theta0
#     for iter in xrange(start_iter + 1, num_iters + 1):
#         _, grad = f(theta)
   
#         # there is NO dot product ! return theta
#         theta = theta - (alpha * grad) 

In [10]:
num_epochs = 5
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000214
epoch 2, loss 0.000099
epoch 3, loss 0.000099
epoch 4, loss 0.000098
epoch 5, loss 0.000098


In [11]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-2.1219e-05, -2.7084e-04])
b的估计误差： tensor([0.0005])
